## ДЗ 2

### Задание 1.
Задание: обучите три классификатора:

1) на токенах с высокой частотой

2) на токенах со средней частотой

3) на токенах с низкой частотой

Сравните полученные результаты, оцените какие токены наиболее важные для классификации.
#### Сделано

### Задание 2.
найти фичи с наибольшей значимостью, и вывести их
#### Сделано

### Задание 3.
1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)
#### Сделано кроме полносвязной сетки

2) подобрать оптимальный размер для hashing векторайзера
#### Сделано

3) убедиться что для сетки нет переобучения
#### Не успела, доделаю на выходных до 24.04


In [61]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import nltk
import re

In [63]:
with open('hw2_corpus.txt', 'r') as f:
    df = pd.DataFrame({'category': t[0], 'text': t[1]} for t in (l.split(' ', 1) for l in f.readlines()))
df['category'] = df['category'].apply(lambda t: re.search(r'\d+', t).group())
df

,category,text
0,2,Stuning even for the non-gamer: This sound tra...
1,2,The best soundtrack ever to anything.: I'm rea...
2,2,Amazing!: This soundtrack is my favorite music...
3,2,Excellent Soundtrack: I truly like this soundt...
4,2,"Remember, Pull Your Jaw Off The Floor After He..."
...,...,...
9995,2,A revelation of life in small town America in ...
9996,2,Great biography of a very interesting journali...
9997,1,Interesting Subject; Poor Presentation: You'd ...
9998,1,Don't buy: The box looked used and it is obvio...


In [64]:
from string import punctuation
stopwords = nltk.corpus.stopwords.words('english')
noise = set(stopwords + list(punctuation))
df['text_tokens'] = df['text'].apply(lambda t: [w for w in nltk.word_tokenize(t) if w not in noise])
df

,category,text,text_tokens
0,2,Stuning even for the non-gamer: This sound tra...,"[Stuning, even, non-gamer, This, sound, track,..."
1,2,The best soundtrack ever to anything.: I'm rea...,"[The, best, soundtrack, ever, anything, I, 'm,..."
2,2,Amazing!: This soundtrack is my favorite music...,"[Amazing, This, soundtrack, favorite, music, t..."
3,2,Excellent Soundtrack: I truly like this soundt...,"[Excellent, Soundtrack, I, truly, like, soundt..."
4,2,"Remember, Pull Your Jaw Off The Floor After He...","[Remember, Pull, Your, Jaw, Off, The, Floor, A..."
...,...,...,...
9995,2,A revelation of life in small town America in ...,"[A, revelation, life, small, town, America, ea..."
9996,2,Great biography of a very interesting journali...,"[Great, biography, interesting, journalist, Th..."
9997,1,Interesting Subject; Poor Presentation: You'd ...,"[Interesting, Subject, Poor, Presentation, You..."
9998,1,Don't buy: The box looked used and it is obvio...,"[Do, n't, buy, The, box, looked, used, obvious..."


In [65]:
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
df['text_tokens'] = df['text_tokens'].apply(lambda l: [ lemmatizer.lemmatize(t) for t in l ])
df

,category,text,text_tokens
0,2,Stuning even for the non-gamer: This sound tra...,"[Stuning, even, non-gamer, This, sound, track,..."
1,2,The best soundtrack ever to anything.: I'm rea...,"[The, best, soundtrack, ever, anything, I, 'm,..."
2,2,Amazing!: This soundtrack is my favorite music...,"[Amazing, This, soundtrack, favorite, music, t..."
3,2,Excellent Soundtrack: I truly like this soundt...,"[Excellent, Soundtrack, I, truly, like, soundt..."
4,2,"Remember, Pull Your Jaw Off The Floor After He...","[Remember, Pull, Your, Jaw, Off, The, Floor, A..."
...,...,...,...
9995,2,A revelation of life in small town America in ...,"[A, revelation, life, small, town, America, ea..."
9996,2,Great biography of a very interesting journali...,"[Great, biography, interesting, journalist, Th..."
9997,1,Interesting Subject; Poor Presentation: You'd ...,"[Interesting, Subject, Poor, Presentation, You..."
9998,1,Don't buy: The box looked used and it is obvio...,"[Do, n't, buy, The, box, looked, used, obvious..."


In [90]:
from collections import Counter
corpus = [ t for ts in df.text_tokens for t in ts ]
freq_dicts = list(tp[0] for tp in sorted(Counter(corpus).items(), key=lambda x: -x[1]))

freq_dicts = { 
    'all': set(freq_dicts),
    '<5%': set(freq_dicts[:len(freq_dicts)//20]), 
    '5-20%': set(freq_dicts[len(freq_dicts)//20 : len(freq_dicts)//5]),  
    '>20%': set(freq_dicts[len(freq_dicts)//5:]),
}


In [104]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer


df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

def filtered_text(df_, freq_type):
    return [ ' '.join(t for t in tt if t in freq_dicts[freq_type]) for tt in df_['text_tokens'] ]
    
def fit_and_test(freq_type, vec, clf=None):
    print('Символы частоты: ' + freq_type)
    x_train = filtered_text(df_train, freq_type)
    y_train = df_train['category']
    x_test = filtered_text(df_test, freq_type)
    y_test = df_test['category']
    
    if clf == None:
        clf = LogisticRegression(random_state=42)
    
    bow = vec.fit_transform(x_train)
    
    clf.fit(bow, y_train)
    if hasattr(vec, 'get_feature_names'):
        feature_names = vec.get_feature_names()
        coefs_with_importances = sorted(zip(clf.coef_[0], feature_names))
        n_important = 10;
        print("Наиболее значимые положительные: ")
        for t in reversed(coefs_with_importances[-n_important:]):
            print(f"{t[1]} : {t[0]:.3f}")
        print("Наиболее значимые отрицательные: ")
        for t in coefs_with_importances[:n_important]:
            print(f"{t[1]} : {t[0]:.3f}")

    pred = clf.predict(vec.transform(x_test))
    print(classification_report(pred, y_test))
    
    

print('CountVectorizer:')
for freq_type in freq_dicts:
    fit_and_test(freq_type, CountVectorizer(ngram_range=(1, 1)))
    
print('TfidfVectorizer:')
for freq_type in freq_dicts:
    fit_and_test(freq_type, TfidfVectorizer(ngram_range=(1, 1)))
    
for n_features in [100, 200, 500, 1000, 10000, 100000, 1000000]:
    print(f'HashingVectorizer with {n_features} features:')
    fit_and_test('all', HashingVectorizer(analyzer='word', n_features=n_features))


CountVectorizer:
Символы частоты: all


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Наиболее значимые положительные: 
excellent : 2.419
perfect : 1.930
amazing : 1.493
awesome : 1.478
government : 1.408
wonderful : 1.343
works : 1.320
love : 1.286
emotion : 1.245
favorite : 1.224
Наиболее значимые отрицательные: 
boring : -2.423
poor : -2.387
waste : -2.072
worst : -2.027
disappointing : -1.814
disappointed : -1.764
not : -1.584
disappointment : -1.499
awful : -1.395
horrible : -1.339
              precision    recall  f1-score   support

           1       0.87      0.86      0.86      1045
           2       0.85      0.86      0.85       955

    accuracy                           0.86      2000
   macro avg       0.86      0.86      0.86      2000
weighted avg       0.86      0.86      0.86      2000

Символы частоты: <5%


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Наиболее значимые положительные: 
excellent : 2.336
intense : 2.083
perfect : 1.982
epic : 1.778
amazing : 1.714
fantastic : 1.634
helped : 1.617
emotion : 1.556
provides : 1.503
flow : 1.498
Наиболее значимые отрицательные: 
boring : -2.878
worst : -2.407
poor : -2.406
disappointment : -2.230
disappointing : -2.070
waste : -2.004
beware : -1.831
misleading : -1.769
terrible : -1.736
ridiculous : -1.716
              precision    recall  f1-score   support

           1       0.83      0.83      0.83      1044
           2       0.81      0.82      0.82       956

    accuracy                           0.82      2000
   macro avg       0.82      0.82      0.82      2000
weighted avg       0.82      0.82      0.82      2000

Символы частоты: 5-20%


...

Наиболее значимые положительные: 
great : 6.640
excellent : 5.203
love : 4.993
best : 4.178
perfect : 3.570
good : 3.564
wonderful : 3.115
amazing : 3.037
easy : 2.884
well : 2.850
Наиболее значимые отрицательные: 
boring : -5.371
not : -4.996
waste : -4.582
poor : -4.551
worst : -4.529
disappointed : -4.256
bad : -3.907
money : -3.550
disappointing : -3.070
do : -2.951
              precision    recall  f1-score   support

           1       0.86      0.84      0.85      1055
           2       0.83      0.84      0.84       945

    accuracy                           0.84      2000
   macro avg       0.84      0.84      0.84      2000
weighted avg       0.84      0.84      0.84      2000

Символы частоты: 5-20%


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Наиболее значимые положительные: 
awsome : 1.768
medical : 1.714
favourite : 1.481
enjoying : 1.455
ya : 1.445
outstanding : 1.431
amazing : 1.426
ease : 1.424
turner : 1.419
enjoys : 1.411
Наиболее значимые отрицательные: 
dont : -2.222
awful : -2.164
wrong : -1.988
false : -1.891
junk : -1.873
annoyed : -1.799
missing : -1.754
none : -1.709
drivel : -1.623
ripped : -1.594
              precision    recall  f1-score   support

           1       0.69      0.71      0.70      1008
           2       0.69      0.67      0.68       992

    accuracy                           0.69      2000
   macro avg       0.69      0.69      0.69      2000
weighted avg       0.69      0.69      0.69      2000

Символы частоты: >20%


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Наиболее значимые положительные: 
heart : 1.601
well : 1.341
must : 1.142
highly : 1.101
rock : 1.095
excelent : 1.025
great : 0.981
provoking : 0.960
own : 0.957
riveting : 0.932
Наиболее значимые отрицательные: 
money : -1.360
re : -1.308
pseudo : -1.089
what : -1.066
harlequin : -1.046
self : -1.032
either : -1.016
00 : -1.013
there : -1.011
half : -0.992
              precision    recall  f1-score   support

           1       0.73      0.60      0.66      1269
           2       0.47      0.62      0.54       731

    accuracy                           0.61      2000
   macro avg       0.60      0.61      0.60      2000
weighted avg       0.64      0.61      0.62      2000

HashingVectorizer with 100 features:
Символы частоты: all
              precision    recall  f1-score   support

           1       0.66      0.66      0.66      1034
           2       0.63      0.63      0.63       966

    accuracy                           0.65      2000
   macro avg       0.65      0.65   

### Выводы по заданию 1:
Наилучший результат получился с использованием наиболее популярных токенов, как с CountVectorizer, так и с TfidfVectorizer. Вероятно, наименее популярные токены хорошо выделяют отдельные документы, но не категорию. Самый лучший результат получался по полному набору токенов

### Выводы по заданию 2:
Фичи с наибольшей значимостью вполне подходят к каждой из категорий, особенно для полного и наиболее популярного наборов

### Выводы по заданию 3:
1. (кроме полносвязной сетки) - лучше всего отработал TfIdfVectorizer. HashingVectorizer приблизился к этому результату на больших размерах > 10000
2. Близкие к максимальным метрики получаются у HashingVectorizer, начиная с размера 10000, но продолжают слегка улучшаться вплоть до 1000000.